# TUTORIAL

In this notebook, we will show how to access the documents in the different VerbCL collections:
* VerbCL Opinions
* VerbCL CitationGraph
* VerbCL Highlights

We access ElasticSearch through the module `elasticsearch-dsl`, refer to the [Documentation](https://elasticsearch-dsl.readthedocs.io/en/latest/) for more information. All the ElasticSearch queries are available through this module.

The documents are described in the file `utils/elastic.py` as the classes `OpinionDocument`, `OpinionCitationGraph` and `OpinionSentence`.

In [ ]:
import sys
sys.path.append('../')

In [ ]:
import pandas as pd

from utils import elastic_init
from utils import OpinionCitationGraph, OpinionSentence, OpinionDocument

from elasticsearch_dsl import connections

In [ ]:
pd.set_option("display.max_colwidth", None)

# Connect to ElasticSearch

Adjust the environment variables in the file `elastic_local.env`

In [ ]:
alias = elastic_init("elastic-local.env")
print(alias)

In [ ]:
OPINION_ID = 1239944

# VerbCL Opinions

We will retrieve the opinion 1239944 used as an example in the paper.

In [ ]:
s = OpinionDocument.search(using=alias).query("match", opinion_id=OPINION_ID)

In [ ]:
assert s.count() == 1

In [ ]:
retrieved = list(s.scan())

In [ ]:
opinion = retrieved[0]

In [ ]:
print(opinion.raw_text)

# VerbCL Citation Graph

We will retrieve all the opinions that cite this opinion 1239944.

In [ ]:
s = OpinionCitationGraph.search(using=alias).query("match", cited_opinion_id=OPINION_ID).filter("range", score={"gt": -1})
scan = s.scan()

In [ ]:
citings = pd.DataFrame([r.to_dict() for r in scan])

In [ ]:
citings

# VerbCL Highlights

We will retrieve all the highlights from opion 1239944.

In [ ]:
s = OpinionSentence.search(using=alias).query("match", opinion_id=OPINION_ID).query("match", highlight=True)
scan = s.scan()

In [ ]:
highlights = pd.DataFrame([r.to_dict() for r in scan])

In [ ]:
highlights[['sentence_id', 'count_citations', 'raw_text']]